# Application d'extractions des relations sociales entre entités nommées

- CHANGER CODE : PAS DE TEXT.SPLIT, REMPLIR LE TABLEAU PAR PHRASE
- UTILISER L'APPOSITION POUR LE PB DE SYMETRIE
- REmplacer les 'her', 'him', 'she', etc... par les noms des entités nommées en utilisant des règles 'à la main'
- Penser aux words-embedding
- Passer par les nmod:pass pour créer les entités et l'apposition pour caractériser la relation

In [144]:
from stanfordcorenlp import StanfordCoreNLP

In [145]:
nlp = StanfordCoreNLP('stanford-corenlp-full-2018-10-05')

In [146]:
file = open('Hamlet.txt','r+')
text = file.read()
lines = text.split('\n')
PERSON = []
DEPENDENCY = [[]]*len(lines)
TOKENS = []
RELATIONSHIP = {}
LINKS = ['son','father','mother','daughter','cousin','siblings','husband','wife','spouses','brother','sister','friend']

In [147]:
nlp.dependency_parse(text)

[('ROOT', 0, 7),
 ('amod', 2, 1),
 ('nmod:npmod', 3, 2),
 ('advmod', 7, 3),
 ('case', 5, 4),
 ('nmod', 3, 5),
 ('expl', 7, 6),
 ('det', 9, 8),
 ('nsubj', 7, 9),
 ('acl', 9, 10),
 ('xcomp', 10, 11),
 ('punct', 7, 12),
 ('ROOT', 0, 10),
 ('nummod', 2, 1),
 ('nmod:tmod', 10, 2),
 ('nmod:poss', 5, 3),
 ('case', 3, 4),
 ('dep', 2, 5),
 ('punct', 10, 6),
 ('det', 8, 7),
 ('nsubj', 10, 8),
 ('punct', 10, 9),
 ('advmod', 10, 11),
 ('cc', 10, 12),
 ('nsubj', 16, 13),
 ('cop', 16, 14),
 ('advmod', 16, 15),
 ('conj', 10, 16),
 ('punct', 10, 17),
 ('ROOT', 0, 11),
 ('case', 2, 1),
 ('nmod', 11, 2),
 ('punct', 11, 3),
 ('nmod:poss', 6, 4),
 ('case', 4, 5),
 ('nsubjpass', 11, 6),
 ('punct', 6, 7),
 ('appos', 6, 8),
 ('punct', 6, 9),
 ('auxpass', 11, 10),
 ('advmod', 11, 12),
 ('advmod', 14, 13),
 ('advmod', 11, 14),
 ('punct', 11, 15),
 ('ROOT', 0, 2),
 ('nsubj', 2, 1),
 ('nmod:poss', 4, 3),
 ('nmod:poss', 6, 4),
 ('case', 4, 5),
 ('nsubj', 15, 6),
 ('punct', 6, 7),
 ('appos', 6, 8),
 ('punct', 6, 9

In [140]:
for line in lines :
    NE = nlp.ner(line)
    TOKENS.append(nlp.word_tokenize(line))
    for elt in NE :
        if 'PERSON' in elt :
            PERSON.append(elt[0])

PERSON = list(set(PERSON))
print(PERSON)

print(DEPENDENCY)

['Hamlet', 'Claudius', 'Horatio', 'Mother', 'Gertrude', 'Ophelia']
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [142]:
nb_line = 0
for line in lines :
    
    for elt in nlp.dependency_parse(line) :
        
        if 'nmod:poss' in elt :
            DEPENDENCY[nb_line] = DEPENDENCY[nb_line] + [(elt,nb_line)]

        elif 'appos' in elt : 
            DEPENDENCY[nb_line] = DEPENDENCY[nb_line] + [(elt,nb_line)]

    nb_line+=1

In [143]:
for elt in DEPENDENCY : 
    for i in elt: 
        print(i)

(('nmod:poss', 5, 3), 1)
(('nmod:poss', 5, 3), 2)
(('nmod:poss', 6, 4), 2)
(('appos', 6, 8), 2)
(('nmod:poss', 4, 3), 2)
(('nmod:poss', 6, 4), 2)
(('appos', 6, 8), 2)
(('nmod:poss', 6, 4), 3)
(('appos', 6, 8), 3)
(('nmod:poss', 4, 3), 3)
(('nmod:poss', 6, 4), 3)
(('appos', 6, 8), 3)
(('nmod:poss', 5, 3), 4)
(('appos', 5, 7), 4)
(('nmod:poss', 8, 6), 4)
(('nmod:poss', 5, 3), 5)
(('appos', 5, 7), 5)
(('nmod:poss', 8, 6), 5)
(('nmod:poss', 11, 10), 7)
(('appos', 11, 13), 7)
(('nmod:poss', 11, 10), 8)
(('appos', 11, 13), 8)
(('nmod:poss', 8, 7), 10)
(('nmod:poss', 12, 10), 10)
(('nmod:poss', 8, 7), 11)
(('nmod:poss', 12, 10), 11)
(('nmod:poss', 15, 13), 14)
(('appos', 15, 17), 14)
(('nmod:poss', 15, 13), 15)
(('appos', 15, 17), 15)
(('nmod:poss', 17, 16), 15)
(('nmod:poss', 17, 16), 16)
(('appos', 5, 7), 18)
(('appos', 5, 7), 19)
(('nmod:poss', 12, 11), 23)
(('nmod:poss', 12, 11), 24)
(('nmod:poss', 5, 4), 24)


In [130]:
nlp.close()

ZombieProcess: psutil.ZombieProcess process still exists but it's a zombie (pid=71933)

On dispose maintenant d'un tableau avec dans chaque colonne toutes les dépendances d'une ligne du texte
Il faut maintenant analyser pour chaque colonne si un des mots de la dépendance est un lien social et qu'il est relié à un prénom (en les comparant à nos listes LINKS et PERSON), et si c'est le cas, l'enregistrer dans un dictionnaire de la forme "lien_social : (A,B)" ou "A : (lien_social,B)"

In [126]:
for elt in DEPENDENCY :
    for i in elt :
        try :
            line = (i[1])
            num1 = (i[0][1])-1
            num2 = (i[0][2])-1
            print('Ligne : ',line)
            print('Mots : ',num1,num2)
            
            word1 = TOKENS[line][num1]
            word2 = TOKENS[line][num2]
            if(word1 in LINKS and word2 in PERSON) :
                
                print(word1, word2, ':', True)
                print('\n')
            
            elif(word1 in PERSON and word2 in LINKS) :
                print(word1, word2, ':', True)
                print('\n')
                
            else : 
                print(False)
                print('\n')

        except : 
            print('ligne',line,'vide \n')



Ligne :  1
Mots :  4 2
father Hamlet : True


Ligne :  2
Mots :  5 3
mother Hamlet : True


Ligne :  2
Mots :  5 7
mother Gertrude : True


Ligne :  2
Mots :  3 2
False


Ligne :  2
Mots :  5 3
mother Hamlet : True


Ligne :  2
Mots :  5 7
mother Gertrude : True


Ligne :  4
Mots :  4 2
friend Hamlet : True


Ligne :  4
Mots :  4 6
friend Horatio : True


Ligne :  4
Mots :  7 5
False


Ligne :  7
Mots :  10 9
False


Ligne :  7
Mots :  10 12
False


Ligne :  10
Mots :  7 6
False


Ligne :  10
Mots :  11 9
False


Ligne :  14
Mots :  14 12
brother Ophelia : True


Ligne :  14
Mots :  14 16
False


Ligne :  15
Mots :  16 15
False


Ligne :  18
Mots :  4 6
False


Ligne :  23
Mots :  11 10
False


Ligne :  24
Mots :  4 3
False




In [127]:
#dictionnary = {}
#dictionnary['Hamlet'] = [('father','The King'),('Mother','Gertrude')]
#print(dictionnary)
#dictionnary['Hamlet'] += [('lool','ff')]
#print(dictionnary)